---
# Comparando regressões e Conclusão do projeto
**Grupo:** Isadora Dalberto, Lais da Silva, Gabriela Moreno e Stephanie Liu

Ao longo do desenvolvimento do projeto foram utilizadas algumas técnicas de regressão, pois o problema envolvia problemas com variáveis quantitativas e continuas, foram usadas para ver a influência de cada variável na felicidade dos países e assim analisar a influência do consumo de álcool no happiness scores, e assim, ver se podem ser feitas hipóteses. As técnicas usadas foram: Regressão Linear, Decision Tree, Random Frorest Regression, e K-Neighbors Regression. Para compararmos o resultado utilizaremos a mesma base de treinamento e teste para que seja possível a comparação das regressões 

Este problema é relevante para governos, pois é possível analisar a tendência ao alcoolismo da população de acordo com as taxas apresentadas e assim realizar ações que sejam eficientes para cada país. Além disso, seria relevante para empresas de bebidas que dependendo dos resultados podem escolher em quais países vale mais a pena investir, pode servir como forma de marketing também, mostrando a bebida pode trazer felicidade ou melhorar o sentimento de tristeza.

---

## Preparando o ambiente 
* Importando biblioteca, lendo dados e separando a base de treinamento e teste  

---

In [9]:
# --------------------- IMPORTANDO BIBLIOTECA ----------------------
%matplotlib inline
#!pip install mglearn
# !pip install pydotpus
# !pip install graphviz
# import pydotplus
import numpy as np
import pandas as pd
import matplotlib
import matplotlib.pyplot as plt
import seaborn as sns
import random
from sklearn.model_selection import train_test_split
import mglearn 
from sklearn.neighbors import KNeighborsRegressor
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor
from sklearn.tree import export_graphviz, DecisionTreeClassifier, DecisionTreeRegressor
from sklearn.externals.six import StringIO  
from IPython.display import Image  
from sklearn.metrics import r2_score

RANDOM_SEED = 42
np.random.seed(RANDOM_SEED)

In [10]:
# --------------- Leitura dos dados ---------------
dados = pd.read_excel("DadosTodasAsVariaveis.xlsx")
dados2 = pd.read_excel("HappinessAlcoholConsumption.xls")
#list(dados)
dados2.head()

,Country,Region,Hemisphere,HappinessScore,HDI,GDP_PerCapita,Beer_PerCapita,Spirit_PerCapita,Wine_PerCapita
0,Denmark,Western Europe,north,7.526,928,53.579,224,81,278
1,Switzerland,Western Europe,north,7.509,943,79.866,185,100,280
2,Iceland,Western Europe,north,7.501,933,60.530,233,61,78
3,Norway,Western Europe,north,7.498,951,70.890,169,71,129
4,Finland,Western Europe,north,7.413,918,43.433,263,133,97


In [11]:
RANDOM_SEED = 42
np.random.seed(RANDOM_SEED)

In [17]:
# --------------- separando as colunas ---------------
X =  dados[['HDI', 'GDP_PerCapita', 'Mortalidade', 'Inflação', 'Criança p/ Mulher',"Expec de Vida","Desemprego","Beer_PerCapita","Spirit_PerCapita","Wine_PerCapita"]]
#X2 = dados[["HDI","GDP_PerCapita","Beer_PerCapita","Spirit_PerCapita","Wine_PerCapita"]]
y = dados["HappinessScore"]

# --------------- criando base de teste e treinamento fixa ---------------
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.1, random_state=RANDOM_SEED)

# --------------- Criando a base de teste e treinamento para o excel --------------- 
# para não salvar em cima 
# base_treinamento = pd.concat([X_train,y_train])
# base_treinamento.to_excel("Base_Treinamento.xlsx",sheet_name="Treinamento")
# base_teste = pd.concat([X_test,y_test])
# base_teste.to_excel("Base_Teste.xlsx",sheet_name="Teste")

# -------- lendo os arquivos em excel ---------- 
treinamento = pd.read_excel("Base_Treinamento.xlsx",sheet_name="Treinamento")
teste = pd.read_excel("Base_Teste.xlsx",sheet_name="Teste")
treinamento.columns=['HDI', 'GDP_PerCapita', 'Mortalidade', 'Inflação', 'Criança p/ Mulher',"Expec de Vida","Desemprego","Beer_PerCapita","Spirit_PerCapita","Wine_PerCapita","Happiness"]
treinamento.head(5)
#teste.columns=['HDI', 'GDP_PerCapita', 'Mortalidade', 'Inflação', 'Criança p/ Mulher',"Expec de Vida","Desemprego","Beer_PerCapita","Spirit_PerCapita","Wine_PerCapita","Happiness"]

,HDI,GDP_PerCapita,Mortalidade,Inflação,Criança p/ Mulher,Expec de Vida,Desemprego,Beer_PerCapita,Spirit_PerCapita,Wine_PerCapita,Happiness
Croatia,828,12.299,4.7,-0.0843,1.46,77.3,13.103400,230,87,254,5.488
Qatar,855,59.324,8.5,-9.4200,1.91,80.4,0.150000,1,42,7,6.375
South Africa,696,5.280,43.3,6.7700,2.46,62.4,26.550699,225,76,81,4.459
Cote d'Ivoire,486,1.535,91.8,1.5700,4.91,60.0,2.599200,37,1,7,3.916
Mexico,772,8.444,14.6,5.3600,2.18,76.4,3.859000,238,68,5,6.778


In [13]:
# --------------- funções para acuracia --------------- 
def Erro(y_previsto,y_base_teste):
    lista_erro = []
    for i,e in zip(y_previsto,y_base_teste):
        if i < e:
            dif = e-i
            lista_erro.append(dif)
        else: 
            dif = i-e
            lista_erro.append(dif)
    return lista_erro

def Classificador(lista_erro):
    lista_classificacao = []
    for erros in lista_erro:
        if erros<0.5:
            classifica = 'Certo'
            lista_classificacao.append(classifica)
        else:
            classifica = 'Errado'
            lista_classificacao.append(classifica)
    return lista_classificacao 

def Acuracia(Classificador,Erro,y_predict,y_test):
    data = {
    'Y_predict': y_predict,
    'Y_test': y_test,
    'Erro': Erro, 
    'Classificação': Classificador,
    }
    tabela = pd.DataFrame(data, columns=['Y_predict', 'Y_test','Erro','Classificação'])
    tabela.head(5)
    contador=0
    for c in Classificador: 
        if c == "Certo": 
            contador+=1
    acertou=(contador/len(Classificador))*100
    return acertou 

---
# Primeira regressão: Regressão Linear 

---

In [14]:
# Selecionando colunas que foram melhores no proprio arquivo de evolução desta regressão 
X_train1 = treinamento[["HDI","Expec de Vida","Desemprego"]]
y_train1 = treinamento["Happiness"]

X_test1 = teste[["HDI","Expec de Vida","Desemprego"]]
y_test1 = teste["Happiness"]

#Fazendo a regressão

#calculando a acuracia da regressão
# erro1 = Erro(y_pred1,y_test1)
# classificando1 = Classificador(erro1)
# acuracia1 = Acuracia(classificando1,erro1,y_pred1,y_test1)
# print("A Regressão Linear acertou {0}%".format(acuracia1))

---
# Segunda regressão: Decision Tree
---

In [15]:
# Selecionando colunas que foram melhores no proprio arquivo de evolução desta regressão
X_train2 = treinamento[["HDI","GDP_PerCapita","Beer_PerCapita","Spirit_PerCapita","Wine_PerCapita"]]
y_train2 = treinamento["Happiness"]

X_test2 = teste[["HDI","GDP_PerCapita","Beer_PerCapita","Spirit_PerCapita","Wine_PerCapita"]]
y_test2 = teste["Happiness"]

#Fazendo a regressão
dt2 = DecisionTreeRegressor(random_state = 42, max_depth=4) 
dt2 = dt2.fit(X_train2,y_train2)

#Prevendo 
y_pred2 = dt2.predict(X_test2)

#calculando a acuracia da regressão
erro2 = Erro(y_pred2,y_test2)
classificando2 = Classificador(erro2)
acuracia2 = Acuracia(classificando2,erro2,y_pred2,y_test2)
print("A Decision Tree acertou {0}%".format(acuracia2))

A Decision Tree acertou 75.0%


---
# Terceira regressão: Random Forest Regression

---

In [8]:
# Selecionando colunas que foram melhores no proprio arquivo de evolução desta regressão
X_train3 = treinamento[["HDI","GDP_PerCapita","Beer_PerCapita","Spirit_PerCapita","Wine_PerCapita"]]
y_train3 = treinamento["Happiness"]

X_test3 = teste[["HDI","GDP_PerCapita","Beer_PerCapita","Spirit_PerCapita","Wine_PerCapita"]]
y_test3 = teste["Happiness"]

#Fazendo a regressão
dt3 = RandomForestRegressor(n_estimators=1000, random_state=RANDOM_SEED)
dt3 = dt3.fit(X_train3,y_train3)

#Prevendo 
y_pred3 = dt3.predict(X_test3)

#calculando a acuracia da regressão
erro3 = Erro(y_pred3,y_test3)
classificando3 = Classificador(erro3)
acuracia3 = Acuracia(classificando3, erro3,y_pred3,y_test3)
print("A Random Forest Regression acertou {0}%".format(acuracia3))

A Random Forest Regression acertou 87.5%


---
# Quarta regressão: KNeighborsRegressor

---

In [13]:
# Selecionando colunas que foram melhores no proprio arquivo de evolução desta regressão(Conclusão foi dada a partir do R^2)

X_train4 = treinamento[["HDI","GDP_PerCapita"]]
y_train4 = treinamento["Happiness"]

X_test4 = teste[["HDI","GDP_PerCapita"]]
y_test4 = teste["Happiness"]

#Fazendo a Regressão K-Neighbors
reg = KNeighborsRegressor(n_neighbors=5)
reg.fit(X_train4, y_train4)

#Prevendo
y_predict4 = reg.predict(X_test4)

#Calculo da acurácia
erro4 = Erro(y_predict4 ,y_test4)
classificando4 = Classificador(erro4)
acuraria4 = Acuracia(classificando4, erro4, y_predict4, y_test4)
print("A regressão Kneighbors acertou {0}%".format(acuraria4))

# R quadrado
r2_score(y_predict4, y_test4)




A regressão Kneighbors acertou 75.0%


0.6895350120226754

---
# Comparando as quatro regressões

ESCREVER

---

---
# Conclusão dos resultados
Caso o estudo sobre esse tema fosse mais afundo, por exemplo com um sociólogo, pode-se obter que o consumo de álcool não influenciou muito na felicidade, porque para lugares felizes as pessoas comemoram então bebem em ocasiões boas, no entanto em países pobres o consumo de álcool também pode ser alto, mas representando um problema de alcoolismos, que é um cenário bem diferente. No entanto as duas opções se anulam e prejudicam a análise. Dessa forma para continuar a pesquisa seria necessário analisar essas diferentes situações em que o álcool é consumido

---